<a href="https://colab.research.google.com/github/RemyLpr/defi_ia/blob/main/data/req/req_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import des modules et des données

In [1]:
import urllib.parse
import requests
import numpy as np
import pandas as pd
import random
import string
import csv
from IPython.display import display
import io
from google.colab import files

In [2]:
df_test = pd.read_csv("test_set.csv")
coeffs_date = pd.read_csv("coeffs_date.csv")
coeffs_computer = pd.read_csv("coeffs_computer.csv")

# Initialisation session API

In [3]:
domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)
user_id = 'e68dad03-0d1a-4038-8766-382b25fe4252'

# Requêtes restantes

In [4]:
r = requests.get(path(f"remaining-requests/{user_id}"))
print("Nombre de requêtes restantes :", r.json())

Nombre de requêtes restantes : 649


# Génération des paramètres

In [5]:
def params_generator(df_test, coeffs_date, coeffs_computer):
  # Pour city, répartition uniforme
  cities = df_test["city"].unique().tolist() # array par défaut
  city = random.choice(cities) # ville choisie
  # Pour date, suivre répartition fichier coeffs_date
  date = random.choices(coeffs_date["date"], coeffs_date["coeff"], k = 1)[0] # liste renvoyée, on prend premier terme car k = 1
  # Pour language, répartition uniforme également (choix par défaut)
  languages = df_test["language"].unique().tolist()
  language = random.choice(languages) # langue choisie
  # Pour mobile, répartition suivant le fichier coeffs_computer
  mobile_prob = coeffs_computer["computer"].tolist()[int(date)] # on sélectionne la proba que mobile = 0 correspondant à la date sélectionnée
  mobile_prob = [mobile_prob, 1 - mobile_prob]
  mobile = random.choices([0, 1], mobile_prob, k = 1)[0]
  # Pour l'instant on utilie un avatar par requête (temporaire)
  avatar = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6)) # génération d'un pseudo aléatoire
  # Définition du dictionnaire des paramètres
  params = {"city" : city, "date" : date, "language" : language, "mobile" : mobile, "avatar_name" : avatar}
  return params

In [11]:
params = params_generator(df_test, coeffs_date, coeffs_computer)
params

{'city': 'vienna',
 'date': 25,
 'language': 'romanian',
 'mobile': 0,
 'avatar_name': '6HVKGQ'}

# Requêtage de l'API

**IMPORTANT** <br>
Avant de requêter il est nécessaire de pull la dernière version du fichier train_set.csv depuis le git et ensuite l'importer manuellement dans Colab. Une fois les requêtes effectuées il faut sauvegarder le fichier à l'emplacement local du git et push. Mettre un commentaire suivant le format : <br> prénom + nb_req total + date + heure

In [12]:
nb_req = int(input("Entrez le nombre de requêtes à exécuter "))
old_df = pd.read_csv("train_set.csv")
df = old_df.copy()
req_count = int(old_df["order_requests"].tolist()[-1]) # on récupère la dernière colonne (order_requests) de la dernière ligne
for i in range(1, nb_req + 1):
  params = params_generator(df_test, coeffs_date, coeffs_computer)
  requests.post(path(f'avatars/{user_id}/{params["avatar_name"]}')) # on doit indiquer à l'API qu'on crée ce nouvel avatar
  r = requests.get(path(f"pricing/{user_id}"), params = params)
  result = r.json() # résultat de la requête au format json
  query_res = pd.DataFrame(result['prices'])
  params_df = pd.DataFrame([result['request']]*len(query_res))
  new_df = pd.concat([query_res, params_df], axis=1)
  new_df["avatar_name"] = params["avatar_name"] # ajout du nom de l'avatar
  new_df["order_requests"] = req_count + i # ajout de la colonne order_requests comme dans le test_set
  # on ajoute les données extraites à notre dataset
  df = pd.concat([df, new_df])
df.to_csv("train_set.csv", index=False) # pour pas écrire la colonne Unnamed : 0
print("Requêtage terminé !")

Entrez le nombre de requêtes à exécuter 500
Requêtage terminé !
